In [1]:
import torch
import sys
import yaml
from torchvision import transforms, datasets
import torchvision
import numpy as np
import os
from sklearn import preprocessing
from torch.utils.data.dataloader import DataLoader

In [2]:
sys.path.append('../')
from models.resnet_base_network import ResNet18

In [3]:
batch_size = 512
# Convert a PIL Image [0.0,1.0]
data_transforms = torchvision.transforms.Compose([transforms.ToTensor()])

In [4]:
config = yaml.load(open("/home/yqliu/PJs/BYOL/config/config.yaml", "r"), Loader=yaml.FullLoader)


In [5]:
train_dataset = datasets.STL10('/home/yqliu/Dataset/STL10', split='train', download=False,
                               transform=data_transforms)

test_dataset = datasets.STL10('/home/yqliu/Dataset/STL10', split='test', download=False,
                               transform=data_transforms)


In [7]:
print("Input shape:", train_dataset[0][0].shape)

Input shape: torch.Size([3, 96, 96])


In [8]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True)

In [9]:
device = 'cpu' #'cuda' if torch.cuda.is_available() else 'cpu'
encoder = ResNet18(**config['network'])

## Projection第一个线性层的输入则为表征提取的输出[in_features]
output_feature_dim = encoder.projetion.net[0].in_features

In [10]:
#load pre-trained parameters
load_params = torch.load(os.path.join('/home/yqliu/Dataset/STL10/runs/resnet-18_80-epochs/checkpoints/model.pth'),
                         map_location=torch.device(torch.device(device)))

## 测试仅仅使用在线网络分支 | 且不包括projection和predictor
if 'online_network_state_dict' in load_params:
    encoder.load_state_dict(load_params['online_network_state_dict'])
    print("Parameters successfully loaded.")

# remove the projection head
## .children返回子模块
encoder = torch.nn.Sequential(*list(encoder.children())[:-1])    
encoder = encoder.to(device)

Parameters successfully loaded.


In [11]:
encoder

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [12]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.linear(x)
    

In [13]:
logreg = LogisticRegression(output_feature_dim, 10)
logreg = logreg.to(device)


In [14]:
def get_features_from_encoder(encoder, loader):
    
    x_train = []
    y_train = []

    # get the features from the pre-trained model
    # 提取预训练模型的特征
    for i, (x, y) in enumerate(loader):
        with torch.no_grad():
            feature_vector = encoder(x)
            x_train.extend(feature_vector)
            y_train.extend(y.numpy())

       
    x_train = torch.stack(x_train)
    y_train = torch.tensor(y_train)
    return x_train, y_train


In [15]:
encoder.eval()

## 得到训练/测试的特征
x_train, y_train = get_features_from_encoder(encoder, train_loader)
x_test, y_test = get_features_from_encoder(encoder, test_loader)

if len(x_train.shape) > 2:
    x_train = torch.mean(x_train, dim=[2, 3])
    x_test = torch.mean(x_test, dim=[2, 3])

## 特征
print("Training data shape:", x_train.shape, y_train.shape)
print("Testing data shape:", x_test.shape, y_test.shape)


Training data shape: torch.Size([5000, 512]) torch.Size([5000])
Testing data shape: torch.Size([8000, 512]) torch.Size([8000])


In [16]:
def create_data_loaders_from_arrays(X_train, y_train, X_test, y_test):
    
    ## 先从numpy转换为Tensor, 在第1维concat
    train = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
    
    ##  先从numpy转换为Tensor, 在第1维concat
    test = torch.utils.data.TensorDataset(X_test, y_test)
    ## 为什么batchSize那么大 --- 512
    test_loader = torch.utils.data.DataLoader(test, batch_size=512, shuffle=False)
    return train_loader, test_loader


In [17]:
## 特征标准化
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train).astype(np.float32)
x_test = scaler.transform(x_test).astype(np.float32)


In [19]:
train_loader, test_loader = create_data_loaders_from_arrays(torch.from_numpy(x_train), y_train, torch.from_numpy(x_test), y_test)


In [20]:
## 3e-4
optimizer = torch.optim.Adam(logreg.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()

eval_every_n_epochs = 10

for epoch in range(200):
#     train_acc = []
    ## 训练
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()        
        
        ## 特征最后用于分类器
        logits = logreg(x)
        predictions = torch.argmax(logits, dim=1)
        
        loss = criterion(logits, y)
        
        loss.backward()
        optimizer.step()
    
    total = 0
    
    ## 测试
    if epoch % eval_every_n_epochs == 0:
        correct = 0
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            
            # 接最后的线性层输入类别结果
            logits = logreg(x)
            ## 使用Rank-1 accuracy 结果
            predictions = torch.argmax(logits, dim=1)
            # num
            total += y.size(0)
            # 类别相同计数器
            correct += (predictions == y).sum().item()
            
        acc = 100 * correct / total
        print (f"Testing accuracy: {np.mean(acc)}")
        
        

Testing accuracy: 59.0875
Testing accuracy: 67.525
Testing accuracy: 68.4875
Testing accuracy: 68.9125
Testing accuracy: 69.325
Testing accuracy: 69.3875
Testing accuracy: 69.7125
Testing accuracy: 69.5875
Testing accuracy: 69.9375
Testing accuracy: 69.6625
Testing accuracy: 70.0125
Testing accuracy: 70.175
Testing accuracy: 70.1125
Testing accuracy: 69.975
Testing accuracy: 70.0
Testing accuracy: 70.05
Testing accuracy: 69.9125
Testing accuracy: 69.85
Testing accuracy: 70.05
Testing accuracy: 69.6875
